# Bibliotecas

In [ ]:
#from google.colab import drive

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm.notebook import tqdm
from sklearn.cluster import KMeans

from sklearn.metrics import silhouette_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
sns.set_theme()

In [ ]:
RANDOM_SEED = 33
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Download CIC IDS 2017

[Este](https://www.unb.ca/cic/datasets/ids-2017.html) conjunto de dados contém informações sobre fluxos de rede, representando tanto o tráfego benigno quanto ataques populares. Para o ambiente de teste desta coleta de dados, uma rede foi configurada para o atacante e uma rede separada foi configurada para as vítimas, esta última contendo firewalls, roteadores, switches, servidores e estações de trabalho em execução em diferentes versões dos sistemas operacionais Windows e Linux. Para gerar o tráfego benigno, os autores utilizaram um sistema desenvolvido por eles mesmos, que então extraiu o comportamento abstrato de 25 usuários com base em diferentes protocolos de aplicação. Os dados foram coletados ao longo de cinco dias de atividade de rede e foram processados para extrair mais de 80 features do conjunto de dados usando a ferramenta CICFlowMeter8.


In [ ]:
# update gdown version
!pip install --upgrade --no-cache-dir gdown

In [ ]:
# !wget http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/MachineLearningCSV.zip -O CIC_IDS_2017.zip
!gdown '1WtbUHBpANHLMVVHuaFr9-pGUyeW6QhdD' -O CIC_IDS_2017.zip

In [ ]:
# !unzip MachineLearningCSV.zip
!unzip ./CIC_IDS_2017.zip

# Carregando os dados

In [ ]:
df_list = []
for file in os.listdir('../../../Dados/MachineLearningCVE/'):
  df_aux = pd.read_csv(f'../../../Dados/MachineLearningCVE/{file}')
  df_list.append(df_aux)
df = pd.concat(df_list, ignore_index=True)

In [ ]:
df.info()

In [ ]:
list(df.columns)[:6]

Algumas colunas tem seus nomes iniciados com espaços ou finalizados com espaços. Vamos remover esses espaços não úteis para ajustar o nome das colunas.

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df.info()

# Limpando os dados

É necessário limpar os dados realizando:
- Descarte de registros duplicados
- Descarte de registros com valores NaN (Not a Number)/ Null / NA (Not Available)
- Evitar registros com valores não finitos. Nesse caso, uma abordagem válida é substituirmos os mesmos pelo maior valor finito presente no dataset.

Registros duplicados

In [ ]:
df[df.duplicated()]

In [ ]:
# Descartando duplicadas
initial_len = df.shape[0]
df = df.drop_duplicates()
print(f'Tamanho inicial: {initial_len}, tamanho final {df.shape[0]} | Descartadas {initial_len - df.shape[0]} duplicadas')

Registros com valores Null/NaN/NA

In [ ]:
df.columns[df.isna().any(axis=0)]

In [ ]:
df[df.isna().any(axis=1)][['Flow Bytes/s']]

In [ ]:
# Descartando registros com valores NaN/Null/NA
initial_len = df.shape[0]
df = df.dropna()
print(f'Tamanho inicial: {initial_len}, tamanho final {df.shape[0]} | Descartados {initial_len - df.shape[0]} registros com valores NA')

In [ ]:
df = df.reset_index(drop=True)

Registros com valores não finitos

In [ ]:
df_columns_isfinite = np.isfinite(df.drop(['Label'], axis='columns')).all(axis=0)
df_columns_isfinite[df_columns_isfinite == False]

In [ ]:
df_rows_isfinite = np.isfinite(df.drop(['Label'], axis='columns')).all(axis=1)
inf_indexes = df_rows_isfinite[df_rows_isfinite == False].index
df.iloc[inf_indexes][['Flow Bytes/s', 'Flow Packets/s', 'Flow Duration']]

In [ ]:
# Evitando registros com valores não finitos
max_finite_flow_packets_per_sec = df[np.isfinite(df['Flow Packets/s'])]['Flow Packets/s'].max()
max_finite_flow_bytes_per_sec = df[np.isfinite(df['Flow Bytes/s'])]['Flow Bytes/s'].max()

df.loc[df['Flow Packets/s'] == np.inf, 'Flow Packets/s'] = max_finite_flow_packets_per_sec
df.loc[df['Flow Bytes/s'] == np.inf, 'Flow Bytes/s'] = max_finite_flow_bytes_per_sec

# Mini análise exploratória

### Quantidade de instâncias benignas x maliciosas

In [ ]:
sns.countplot(data=df['Label'].apply(lambda label: 'Malicious' if label != 'BENIGN' else 'Benign').to_frame(), x='Label')

**Dados não balanceados**. Impactos:
- Dificuldade de treinar modelos supervisionados
- Dificuldade de avaliar resultados com métricas tradicionais como acurácia

### Quantidade de instâncias por tipo de ataque

Abaixo está um descritivo para os ataques do dataset:

**DoS (Denial of Service)**: Esses ataques, como "DoS Hulk", "DoS GoldenEye", "DoS Slowloris", "DoS Slowhttptest" e "DDoS" visam tornar temporariamente uma máquina ou recurso de rede indisponível, sendo diferenciados pelo protocolo e estratégia usados para causar a negação de serviço. No caso do "DDoS", várias máquinas Windows 8.1 foram usadas para enviar solicitações UDP, TCP e HTTP.

**FTP Patator" e "SSH Patator**: Usam o software Patator para adivinhar senhas por força bruta com o uso de listas de palavras.

**Web - Brute Force**: Usa força bruta em uma aplicação com listas de palavras.

**Web - Injeção de SQL**: Esse ataque explora vulnerabilidades em máquinas conectadas publicamente à Internet usando injeção SQL.

**Web - XSS (Cross-Site Scripting)**: Representa injeções de scripts em aplicativos da web, visando a execução de ações maliciosas por outros usuários do aplicativo.

**PortScan**: Realizados com a ferramenta NMap, esses ataques buscam informações sobre os serviços e portas abertas em um alvo.

**Bot**: Esse ataque tem várias possibilidades, como roubo de dados, envio de spam e acesso ao dispositivo. .

**Infiltration**: Baseado na infecção de uma máquina após um usuário abrir um arquivo malicioso.

In [ ]:
df['Label'] = df['Label'].replace({'Web Attack � Brute Force':'Brute Force', 'Web Attack � XSS':'XSS', 'Web Attack � Sql Injection':'Sql Injection'})

In [ ]:
sns.countplot(data=df.query('Label != "BENIGN"')[['Label']], x='Label', order = df.query('Label != "BENIGN"')['Label'].value_counts().index)
plt.xticks(rotation=45)

Ataques menos representados

In [ ]:
N_LESS_REPRESENTED_LABELS = 5

sns.countplot(data=df[df['Label'].isin(df.groupby('Label').size().sort_values(ascending=False)[(-1)*N_LESS_REPRESENTED_LABELS:].index)], x='Label')
plt.xticks(rotation=45)

### Estatísticas dos dados

In [ ]:
interesting_cols = ['Flow Duration', 'Flow Bytes/s', 'Total Fwd Packets', 'Average Packet Size', 'SYN Flag Count']
df[interesting_cols].describe()

# Dividindo dados nos conjuntos de treino, validação e teste

**Conjunto de treino**

Para a detecção de anomalias, vamos usar somente os dados que representam o tráfego benigno para o conjunto de treino. Dessa forma, os algoritmos de clustering vão ser capazes de identificar padrões e desvios em relação ao comportamento normal (benigno) dos dados.

**Conjuntos de validação e teste**

Porém, devem ser incluídos dados que representam o tráfego maliciosos nos conjuntos de validação e teste. Esses dados maliciosos no conjunto de validação são importantes para que possamos definir um *threshold* para que seja possível detectar anomalias. Além disso, os dados maliciosos também precisam ser incluídos no conjunto de teste para que possamos avaliar o desempenho do nosso modelo.

In [ ]:
df_train = df.query('Label == "BENIGN"').sample(frac=0.6, random_state=RANDOM_SEED)
df_val_test = df.drop(df_train.index)

df_train = df_train.reset_index(drop=True)
df_val_test = df_val_test.reset_index(drop=True)

X_train = df_train.drop('Label', axis='columns')

In [ ]:
X_val, X_test, classes_val, classes_test = train_test_split(df_val_test.drop('Label', axis='columns'), df_val_test['Label'], test_size=0.65, stratify=df_val_test['Label'], random_state=RANDOM_SEED)

X_val, X_test = X_val.reset_index(drop=True), X_test.reset_index(drop=True)
classes_val, classes_test =  classes_val.reset_index(drop=True), classes_test.reset_index(drop=True)

y_val, y_test = classes_val.apply(lambda c: 0 if c == 'BENIGN' else 1), classes_test.apply(lambda c: 0 if c == 'BENIGN' else 1)

In [ ]:
del df_train, df_val_test

# Analisando correlação entre features

**Por que remover features?**

Vamos descartar features com alta correlação evitando passar informações redundantes ao modelo. Dessa forma, conseguiremos obter um modelo mais simples e com menor custo computacional.

In [ ]:
def get_highly_correlated_features(correlation_matrix, threshold):
  correlated_pairs = []
  for i in range(len(correlation_matrix.columns)):
    for j in range(i):
      if abs(correlation_matrix.iloc[i, j]) > threshold:
        pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
        coefficient = correlation_matrix.iloc[i, j]
        correlated_pairs.append((pair, coefficient))
  return sorted(correlated_pairs, key= lambda pair: pair[1], reverse=True)


In [ ]:
corr_matrix = X_train.corr().abs()
correlation_list = get_highly_correlated_features(corr_matrix, 0.95)

In [ ]:
correlation_list[:10]

In [ ]:
# Drop high correlated features in correlation list

f2drop = []
for feature_pair, _ in correlation_list:
  if feature_pair[0] not in f2drop and feature_pair[1] not in f2drop:
    f2drop.append(feature_pair[1])

In [ ]:
f2drop

A feature "Destination Port", também não fornece muita contribuição devido que a mesma está codificada com valores inteiros, indicando uma relação de grandeza, como 44720 > 80, que não apresenta sentido semântico quando se trata da porta de destino de um fluxo de rede.

In [ ]:
f2drop = f2drop + ['Destination Port']

In [ ]:
X_train = X_train.drop(f2drop, axis='columns')
X_val = X_val.drop(f2drop, axis='columns')
X_test = X_test.drop(f2drop, axis='columns')

# Normalizando os dados

É importante normalizar os dados para lidar com diferentes escalas, sensibilidades a escalas e até mesmo melhorar o desempenho da convergência dos algoritmos.

Caso não seja realizada a normalização, um valor de 10000 para uma feature como "Flow Bytes/s" terá impacto similar ao modelo quanto um valor de 10000 para uma feature como "Flow Packets/s". Isso é prejudicial, pois o impacto desse valor para as duas features deve ser tratado de forma distinta, já que as mesmas têm escalas e sensibilidades também distintas.

In [ ]:
minmax_scaler = MinMaxScaler()
minmax_scaler = minmax_scaler.fit(X_train)

norm_X_train = minmax_scaler.transform(X_train)
norm_X_val = minmax_scaler.transform(X_val)
norm_X_test = minmax_scaler.transform(X_test)

In [ ]:
del X_train, X_val, X_test

# Detecção de Anomalias com Autoencoders

## Autoencoder - Explicação

<div align="center">

![Autoencoder](https://tikz.net/janosh/autoencoder.png)
</div>

Um autoencoder é uma arquitetura de rede neural que aprende a codificar dados em uma representação compacta, chamada de espaço latente, e então reconstruir os dados a partir dessa representação. Ele consiste em duas partes principais: o encoder, que mapeia os dados de entrada para o espaço latente, e o decoder, que reconstrói os dados a partir dessa representação. A ideia central é forçar o modelo a aprender uma representação eficiente e informativa dos dados de entrada.

Os autoencoders são frequentemente usados para tarefas de redução de dimensionalidade e denoising. No entanto, eles também são aplicáveis à detecção de anomalias. A lógica é que um autoencoder treinado em dados normais aprenderá a representação latente desses dados, e quando apresentado com dados anômalos que diferem significativamente dos dados normais, a reconstrução será prejudicada, levando a um erro de reconstrução maior.

## Relembrando o processo de treinamento de redes neurais - Backpropagation

<div align="center">

![Backpropagation](https://miro.medium.com/v2/resize:fit:640/format:webp/1*VF9xl3cZr2_qyoLfDJajZw.gif)
</div>

## Mecanismo de Early Stopping


<div align="center">

![Early stopping](https://www.researchgate.net/publication/356747729/figure/fig3/AS:1098404738408449@1638891505126/Early-stopping-training-is-stopped-as-soon-as-the-performance-on-the-validation-loss.jpg)
</div>

O mecanismo de Early Stopping é uma técnica usada durante o treinamento de redes neurais para evitar overfitting e melhorar a eficiência do modelo. O objetivo é interromper o treinamento assim que a performance do modelo em um conjunto de validação começa a piorar, em vez de continuar até que o desempenho no conjunto de treinamento seja perfeito.

O mesmo inclui usa dos seguintes argumentos para definir um critério de parada:
- **paciência**: Quantidade de épocas limite para esperar melhoria na loss de validação
- **delta**: Melhoria mínina necessária para atualizar uma loss de validação

In [ ]:
# Implementação do Early Stopping
class EarlyStopping:
  def __init__(self, patience=7, delta=0, verbose=True, path='checkpoint.pt'):
      self.patience = patience
      self.delta = delta
      self.verbose = verbose
      self.counter = 0
      self.early_stop = False
      self.val_min_loss = np.Inf
      self.path = path

  def __call__(self, val_loss, model):
    if val_loss < self.val_min_loss - self.delta:   # Caso a loss da validação reduza, vamos salvar o modelo e nova loss mínima
      self.save_checkpoint(val_loss, model)
      self.counter = 0
    else:                                           # Caso a loss da validação NÃO reduza, vamos incrementar o contador da paciencia
      self.counter += 1
      print(f'EarlyStopping counter: {self.counter} out of {self.patience}. Current validation loss: {val_loss:.5f}')
      if self.counter >= self.patience:
          self.early_stop = True

  def save_checkpoint(self, val_loss, model):
    if self.verbose:
        print(f'Validation loss decreased ({self.val_min_loss:.5f} --> {val_loss:.5f}).  Saving model ...')
    torch.save(model, self.path)
    self.val_min_loss = val_loss

## Autoencoder - Implementação

A estrutura básica do autoencoder que vamos montar é:

*features de entrada -> 25 -> 10 -> 25 -> features de entrada*

In [72]:
# Implementação do Autoencoder
class Autoencoder(nn.Module):
  def __init__(self, in_features, dropout_rate=0.2, num_layers=4, tamanho_inicial_camada=25, mltply_en_layer_size=0.4):
    super().__init__()

    self.in_features = in_features
    self.dropout_rate = dropout_rate
    self.early_stopping = None
    self.num_layers = num_layers
    self.tamanho_inicial_camada = tamanho_inicial_camada

    # Lista para armazenar as camadas do encoder
    encoder_layers = []

    # Adiciona camadas ao encoder
    for i in range(num_layers//2):
        if i == 0:
            encoder_layers.append(nn.Linear(in_features, tamanho_inicial_camada))
            tamanho_camada = tamanho_inicial_camada
            encoder_layers.append(nn.BatchNorm1d(tamanho_camada))
            encoder_layers.append(nn.ReLU())
            encoder_layers.append(nn.Dropout(dropout_rate))
        elif i == (num_layers//2 - 1):
            encoder_layers.append(nn.Linear(tamanho_camada, int(tamanho_camada*mltply_en_layer_size)))
            tamanho_final_camada = int(tamanho_camada*mltply_en_layer_size)
            encoder_layers.append(nn.BatchNorm1d(tamanho_final_camada))
            encoder_layers.append(nn.ReLU())
        else:
            encoder_layers.append(nn.Linear(tamanho_camada, int(tamanho_camada*mltply_en_layer_size)))
            tamanho_camada = int(tamanho_camada*mltply_en_layer_size)
            encoder_layers.append(nn.BatchNorm1d(tamanho_camada))
            encoder_layers.append(nn.ReLU())
            encoder_layers.append(nn.Dropout(dropout_rate))

    # Define o encoder como uma sequência
    self.encoder = nn.Sequential(*encoder_layers)


    # Lista para armazenar as camadas do decoder
    decoder_layers = []
    mltply_de_layer_size = 1/mltply_en_layer_size

    # Adiciona camadas ao decoder
    for i in range(num_layers//2):
        if i == 0:
            decoder_layers.append(nn.Linear(tamanho_final_camada, math.ceil(int(tamanho_final_camada*mltply_de_layer_size))))
            tamanho_camada = math.ceil(int(tamanho_final_camada*mltply_de_layer_size))
            decoder_layers.append(nn.BatchNorm1d(tamanho_camada))  # Alterado aqui
            decoder_layers.append(nn.ReLU())
            decoder_layers.append(nn.Dropout(dropout_rate))
        elif i == (num_layers//2 - 1):
            decoder_layers.append(nn.Linear(tamanho_camada, in_features))  # Alterado aqui
            decoder_layers.append(nn.BatchNorm1d(in_features))  # Alterado aqui
            decoder_layers.append(nn.Sigmoid())
        else:
            decoder_layers.append(nn.Linear(tamanho_camada, math.ceil(int(tamanho_camada*mltply_de_layer_size))))
            tamanho_camada = math.ceil(int(tamanho_camada*mltply_de_layer_size))
            decoder_layers.append(nn.BatchNorm1d(tamanho_camada))
            decoder_layers.append(nn.ReLU())
            decoder_layers.append(nn.Dropout(dropout_rate))


    # Define o decoder como uma sequência
    self.decoder = nn.Sequential(*decoder_layers)

  def forward(self, X):
    encoded = self.encoder(X)
    decoded = self.decoder(encoded)
    return decoded

  def compile(self, learning_rate):
    self.criterion = nn.MSELoss()
    self.optimizer = optim.Adam(self.parameters(), lr = learning_rate)

  def fit(self, X_train, num_epochs, batch_size, X_val = None, patience = None, delta = None):
    if X_val is not None and patience is not None and delta is not None:
      print(f'Using early stopping with patience={patience} and delta={delta}')
      self.early_stopping = EarlyStopping(patience, delta)

    val_avg_losses = []
    train_avg_losses = []

    for epoch in range(num_epochs):
      # Calibrando os pesos do modelo
      train_losses = []
      self.train()
      for batch in tqdm(range(0, len(X_train), batch_size)):
        batch_X = X_train[batch:(batch+batch_size)]
        batch_reconstruction = self.forward(batch_X)

        train_loss = self.criterion(batch_reconstruction, batch_X)
        self.optimizer.zero_grad()
        train_loss.backward()
        self.optimizer.step()
        train_losses.append(train_loss.item())
      train_avg_loss = np.mean(train_losses)
      train_avg_losses.append(train_avg_loss)
      print(f'Epoch#{epoch+1}: Train Average Loss = {train_avg_loss:.5f}')

      # Mecanismo de early stopping
      if self.early_stopping is not None:
        val_losses = []
        self.eval()
        with torch.no_grad():
          for batch in range(0, len(X_val), batch_size):
            batch_X = X_val[batch:(batch+batch_size)]
            batch_reconstruction = self.forward(batch_X)
            val_loss = self.criterion(batch_reconstruction, batch_X)
            val_losses.append(val_loss.item())
        val_avg_loss = np.mean(val_losses)
        val_avg_losses.append(val_avg_loss)
        self.early_stopping(val_avg_loss, self)
        if self.early_stopping.early_stop:
          print(f'Stopped by early stopping at epoch {epoch+1}')
          break

    if self.early_stopping is not None:
      self = torch.load('checkpoint.pt')
    self.eval()
    return train_avg_losses, val_avg_losses

In [ ]:
BATCH_SIZE = 256
LR = 5e-4
PATIENCE = 2
DELTA = 0.001
NUM_EPOCHS = 3
IN_FEATURES = norm_X_train.shape[1]
DROPOUT_RATE = 0.2
NUM_LAYERS = 4 # Digite um número par, se for ímpar será truncado para baixo. Será simétrico, ou seja, metade pro encoder e metade pro decoder
INITIAL_LAYER_SIZE = 25 # Tamanho da primeira camada do encoder(não conta com a camada de entrada)
MULTIPLIER_ENCODE_LAYER_SIZE = 0.4 # Multiplicador usado para diminuir e aumentar a quantidade de camadas a partir do INITIAL_LAYER_SIZE, lembrando, o tamanho das camadas sempre tem que ser um inteiro, então, valores quebrados no encoder são truncados pra baixo, e no decoder são truncados pra cima(ceil)

In [73]:
ae_model = Autoencoder(IN_FEATURES, DROPOUT_RATE, NUM_LAYERS, INITIAL_LAYER_SIZE, MULTIPLIER_ENCODE_LAYER_SIZE)
ae_model.compile(learning_rate = LR)

In [74]:
from torchsummary import summary
summary(ae_model, (IN_FEATURES,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 73]           3,942
       BatchNorm1d-2                   [-1, 73]             146
              ReLU-3                   [-1, 73]               0
           Dropout-4                   [-1, 73]               0
            Linear-5                   [-1, 31]           2,294
       BatchNorm1d-6                   [-1, 31]              62
              ReLU-7                   [-1, 31]               0
            Linear-8                   [-1, 71]           2,272
       BatchNorm1d-9                   [-1, 71]             142
             ReLU-10                   [-1, 71]               0
          Dropout-11                   [-1, 71]               0
           Linear-12                   [-1, 53]           3,816
      BatchNorm1d-13                   [-1, 53]             106
          Sigmoid-14                   

In [ ]:
# Exemplo de treinamento sem utilizar Early Stopping

train_avg_losses, _ = ae_model.fit(torch.FloatTensor(norm_X_train), NUM_EPOCHS, BATCH_SIZE)

In [ ]:
# Exemplo de treinamento utilizando Early Stopping

# Passo 1: Considerar apenas amostras benignas no conjunto de validação
benign_norm_X_val = norm_X_val[y_val == 1]
benign_norm_X_val = torch.FloatTensor(benign_norm_X_val)

# Passo 2: Realizar treinamento do modelo
NUM_EPOCHS = 10
ae_model_with_es = Autoencoder(IN_FEATURES)
ae_model_with_es.compile(learning_rate = LR)
train_avg_losses, val_avg_losses = ae_model_with_es.fit(torch.FloatTensor(norm_X_train),
                                                NUM_EPOCHS,
                                                BATCH_SIZE,
                                                X_val = benign_norm_X_val,
                                                patience=PATIENCE,
                                                delta=DELTA)

Abaixo podemos ver um gráfico que exibe as losses (perdas) de treino e validação ao longo das épocas de treinamento. As losses são medidas que indicam quão bem o modelo está aprendendo a tarefa específica para a qual foi treinado. Esse gráfico nos possibilita:

- Acompanhar o treinamento
- Detectar overfitting
- Visualizar a convergência do modelo

In [ ]:
def plot_train_val_losses(train_avg_losses, val_avg_losses):
  epochs = list(range(1, len(train_avg_losses)+1))
  plt.plot(epochs, train_avg_losses, color='blue', label='Loss do treino')
  plt.plot(epochs, val_avg_losses, color='orange', label='Loss da validação')
  plt.title('Losses de treino e validação por época de treinamento')
  plt.legend()

plot_train_val_losses(train_avg_losses, val_avg_losses)

# Definindo um threshold e avaliando resultados

In [ ]:
def plot_roc_curve(y_true, y_score, max_fpr=1.0):
  fpr, tpr, thresholds = roc_curve(y_true, y_score)
  aucroc = roc_auc_score(y_true, y_score)
  plt.plot(100*fpr[fpr < max_fpr], 100*tpr[fpr < max_fpr], label=f'ROC Curve (AUC = {aucroc:.4f})')
  plt.xlim(-2,102)
  plt.xlabel('FPR (%)')
  plt.ylabel('TPR (%)')
  plt.legend()
  plt.title('ROC Curve and AUCROC')

In [ ]:
def get_tpr_per_attack(y_labels, y_pred):
  aux_df = pd.DataFrame({'Label':y_labels,'prediction':y_pred})
  total_per_label = aux_df['Label'].value_counts().to_dict()
  correct_predictions_per_label = aux_df.query('Label != "BENIGN" and prediction == True').groupby('Label').size().to_dict()
  tpr_per_attack = {}
  for attack_label, total in total_per_label.items():
    if attack_label == 'BENIGN':
      continue
    tp = correct_predictions_per_label[attack_label] if attack_label in correct_predictions_per_label else 0
    tpr = tp/total
    tpr_per_attack[attack_label] = tpr
  return tpr_per_attack

In [ ]:
def get_overall_metrics(y_true, y_pred):
  tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
  acc = (tp+tn)/(tp+tn+fp+fn)
  tpr = tp/(tp+fn)
  fpr = fp/(fp+tn)
  precision = tp/(tp+fp)
  f1 = (2*tpr*precision)/(tpr+precision)
  return {'acc':acc,'tpr':tpr,'fpr':fpr,'precision':precision,'f1-score':f1}

In [ ]:
def plot_confusion_matrix(y_true, y_pred):
  cm = confusion_matrix(y_true, y_pred)
  group_counts = [f'{value:.0f}' for value in confusion_matrix(y_true, y_pred).ravel()]
  group_percentages = [f'{value*100:.2f}%' for value in confusion_matrix(y_true, y_pred).ravel()/np.sum(cm)]
  labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_counts, group_percentages)]
  labels = np.array(labels).reshape(2,2)
  sns.heatmap(cm, annot=labels, cmap='Oranges', xticklabels=['Predicted Benign', 'Predicted Malicious'], yticklabels=['Actual Benign', 'Actual Malicious'], fmt='')
  return

## Conjunto de validação

In [ ]:
def get_autoencoder_anomaly_scores(ae_model, X):
  X = torch.FloatTensor(X)
  reconstructed_X = ae_model(X)
  anomaly_scores = torch.mean(torch.pow(X - reconstructed_X, 2), axis=1).detach().numpy() # MSELoss
  return anomaly_scores

In [ ]:
val_anomaly_scores = get_autoencoder_anomaly_scores(ae_model, norm_X_val)

In [ ]:
plot_roc_curve(y_val, val_anomaly_scores)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_val, val_anomaly_scores)
df_val_roc = pd.DataFrame({'fpr':fpr, 'tpr':tpr, 'thresholds':thresholds})
df_val_roc['youden-index'] = df_val_roc['tpr'] - df_val_roc['fpr']
df_val_roc.sort_values('youden-index', ascending=False).drop_duplicates('fpr').query('fpr < 0.03')

In [ ]:
BEST_VALIDATION_THRESHOLD = 0.018680

In [ ]:
plot_confusion_matrix(y_val, val_anomaly_scores > BEST_VALIDATION_THRESHOLD)

In [ ]:
get_overall_metrics(y_val, val_anomaly_scores > BEST_VALIDATION_THRESHOLD)

In [ ]:
get_tpr_per_attack(classes_val, val_anomaly_scores > BEST_VALIDATION_THRESHOLD)

## Conjunto de teste

In [ ]:
test_anomaly_scores = get_autoencoder_anomaly_scores(ae_model, norm_X_test)

In [ ]:
plot_roc_curve(y_test, test_anomaly_scores)

In [ ]:
plot_confusion_matrix(y_test, test_anomaly_scores > BEST_VALIDATION_THRESHOLD)

In [ ]:
get_overall_metrics(y_test, test_anomaly_scores > BEST_VALIDATION_THRESHOLD)

In [ ]:
get_tpr_per_attack(classes_test, test_anomaly_scores > BEST_VALIDATION_THRESHOLD)

# Pergunta e atividade

## Como um autoencoder é capaz de realizar detecção de anomalias?

Insira sua resposta aqui

# Atividade de código

Autoencoders podem ter diferentes arquiteturas de redes neurais para funcionar. O autoencoder visto acima possui as seguintes camadas (representadas juntamente com suas respectivas quantidades de neurônios):

*features de entrada -> 25 neurônios -> 10 neurônios -> 25 neurônios -> 10 neurônios*

**Crie você mesmo e avalie resultados de um autoencoder com uma nova arquitetura, considerando as seguintes camadas (representadas juntamente com suas respectivas quantidades de neurônios):**

**features de entrada -> 30 neurônios -> 20 neurônios -> 10 neurônios -> 20 neurônios -> 30 neurônios -> features de entrada**



OBS: Não é necessário tunar os hiperparâmetros

In [ ]:
# Insira seu código aqui
